In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
titles_corpus = []
dids = []
didToLine = {}
titles=[]
with open("normal_titles.txt") as f:
    for i,line in enumerate(f):
        line = line.strip()
        did,title = line.split("\t")
        did = int(did)
        words = title.split(" ")
        titles.append(title)
        titles_corpus.append(words)
        dids.append(did)
        didToLine[did] = i

In [ ]:
qidToTopTitle = {}
with open("top_titles2.txt") as f:
    for i,line in enumerate(f):
     
      line = line.strip()
      parts = line.split(maxsplit=1)
      if len(parts)==2:
          qid = int(parts[0])
          title = parts[1]
          qidToTopTitle[qid] = title.lower()

In [ ]:
queries_corpus = []
qids = []
qidToLine = {}
queries = []
with open("normal_queries.txt") as f:
    for line in f:
        line = line.strip()
        qid,title = line.split("\t")
        qid = int(qid)
        words = title.split(" ")
        queries.append(title)
        queries_corpus.append(words)
        qidToLine[qid] = len(qids)
        qids.append(qid)
        

In [ ]:
def getQuery(did):
    global queries
    global qidToLine
    if qid in  qidToLine:
        return queries[qidToLine[qid]]
    return ""
def getTitle(did):
    global titles
    global didToLine
    if did in  didToLine:
        return titles[didToLine[did]]
    return ""
def getTitlesForQuery(qid):
    global titles
    global didToLine
    global allQidToDocs
    rv=[]
    dids = []
    if qid in allQidToDocs:
        for did in allQidToDocs[qid]:
            if did in  didToLine:
                rv.append(titles[didToLine[did]])
                dids.append(did)
    return rv, dids  

In [ ]:
import operator
from collections import defaultdict
import numpy as np
def parseTrainTestFile(train_marks_filename, isTest=False):

    trainDidToLine = {}
    qidToDocs = defaultdict(list)
    
    with open(train_marks_filename) as f:
        i=0
        nlines = 0
        for line in f:
            nlines+=1
        
       
        qids = np.zeros( nlines, dtype=int)
        i=0
        f.seek(0)
        if isTest:
            f.readline()
        for ii,line in enumerate(f):
            if not isTest:
                parts = line.split("\t")
                if len(parts)!=3:
                    continue
                qid, did, mark = parts
                qid, did, mark = int(qid), int(did), int(mark)
            else:
                parts = line.split(",")
                if len(parts)!=2:
                    continue
                qid, did = parts
                mark=-1
                qid, did, mark = int(qid), int(did), int(mark)

            qids[i] = qid
            trainDidToLine[(qid,did)] = i

            qidToDocs[qid].append(did)
            i+=1
    return qids, trainDidToLine, qidToDocs
q_train, trainDidToLine, trainQidToDocs = parseTrainTestFile("train.marks.tsv", 
                                                    isTest=False, 
                                                    )

q_test, testDidToLine, testQidToDocs = parseTrainTestFile("sample.csv", 
                                                    isTest=True, 
                                                    )
def url_normalize(url):
    url = url.strip()
    url = url.replace("https://", "")
    url = url.replace("http://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        url = url.replace(":80", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
    return url
def extract_domain(url):
    global rubrics_list
    url = url.strip()
    #for r in rubrics_list:
        #url = url.replace("," + r + "," , "")    
    #    url = url.replace("," + r, "") 
    url = url.replace("https://", "")
    url = url.replace("http://", "")
   
    url = url.replace("ftp://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        
        url = url.replace("s://", "")
        url = url.replace(":80", "")
        url = url.replace("://", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
        pos = url.find("#")
        if pos>0:
            url = url[0:pos]

        pos = url.find("/")
        if pos>0:
            url = url[0:pos]
    return url
def loadUrls(filename):
    urlToDid = {}
    didToUrl = {}
    domainToDid = defaultdict(list)
    didToDomain = defaultdict(list)
   
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            uid, url = line.split("\t")
            uid = int(uid)
            #urls["http://" + url] = uid
            #urls["http://" + url + "/"] = uid
            urlToDid[url_normalize(url)] = uid
            didToUrl[uid] = url_normalize(url)
            domain = extract_domain(url)
            domainToDid[domain].append(uid)
            didToDomain[uid] = domain
            
    return urlToDid,didToUrl,domainToDid,didToDomain
urlToDid,didToUrl,domainToDid,didToDomain = loadUrls("url.data")
urls = urlToDid

In [ ]:
allQidToDocs = defaultdict(list)
didToQueries = defaultdict(list)
for qid,docs in trainQidToDocs.items():
    allQidToDocs[qid]+=(docs)
    for doc in docs:
        didToQueries[doc].append(qid)
for qid,docs in testQidToDocs.items():
    allQidToDocs[qid]+=(docs)
    for doc in docs:
        didToQueries[doc].append(qid)

In [ ]:
cnt=0
not_matched = 0
nqueries=0
found_docs = {}
found_domains = {}
def loadTopDocsFile(filename):
    found_docs = {}
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            qid,fd = line.split(",")
            qid = int(qid)
            fd = int(fd)
            found_docs[qid] = (fd,0.0)
    return found_docs
found_docs = loadTopDocsFile("first_top_docs.txt")
found_domains = loadTopDocsFile("first_top_domains.txt")

In [ ]:
!pip install -U bert-serving-server[http]

  Using cached https://files.pythonhosted.org/packages/b0/bd/cab677bbd0c5fb08b72e468371d2bca6ed9507785739b4656b0b5470d90b/bert_serving_server-1.10.0-py3-none-any.whl
     |████████████████████████████████| 358kB 10.2MB/s 
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5273 sha256=710824ebe4826b213801caf96e11afa60b3b364a38effa79acc914306ebe14d2
  Stored in directory: /root/.cache/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
Successfully built flask-compress


In [ ]:
#дальше скачайте файлы моделей, запустите BERT сервер и выполните этот ноутбук для каждой модели по отдельности

In [ ]:
!pip install bert-serving-client

In [ ]:
questions = queries

In [ ]:
import numpy as np

from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
a = bc.encode(['First do it', 'then do it right', 'then do it better'])
a

array([[ 0.49155793,  0.08796163,  0.08263294, ...,  1.0980438 ,
         0.411266  , -0.2539694 ],
       [-0.07413446, -0.22783582, -0.08978209, ...,  1.5093104 ,
         1.3512006 , -0.03158369],
       [-0.2657499 ,  0.1913579 , -0.35613576, ...,  1.3962182 ,
         1.4187475 ,  0.12651818]], dtype=float32)

In [ ]:
all_qids = [qid for qid in allQidToDocs]

In [ ]:
len(all_qids)

6311

In [ ]:
def getContentForQid(qid):
    print("Getting content for ", qid)
    dids = []
    contents= []
    try:
        with open("/content/content_by_queries/" + str(qid) + ".txt") as f:
            for line in f:
                parts = line.split("\t")
                if len(parts)>=3:
                    did, title, content = int(parts[0]), parts[1], parts[2]
                    dids.append(did)
                    contents.append(content)
        return contents, dids
    except Exception as e:
        #print(repr(e))
        pass

In [ ]:
with open("/content/drive/My Drive/LongTailRelevance/bert_res/BERT_out4_fixed.txt", "w") as f:
  flag=True
  for qid in all_qids:
    try:
      if(flag):
        qtitles,dids  = getTitlesForQuery(qid)
        query = getQuery(qid)
        top_score = np.zeros(len(dids))
        top_score2 = np.zeros(len(dids))
        title_emb =  bc.encode(qtitles)
        if qid in found_docs:
          top_did = found_docs[qid][0]
          top_title = getTitle(top_did)  
          top_emb =  bc.encode([top_title])[0]
          
          top_score =  np.sum(top_emb * title_emb, axis=1) / np.linalg.norm(title_emb, axis=1)
        if qid in qidToTopTitle:
         
          top_title = qidToTopTitle[qid] 
          top_emb =  bc.encode([top_title])[0]
          top_score2 = np.sum(top_emb * title_emb, axis=1) / np.linalg.norm(title_emb, axis=1)
       
        query_emb =  bc.encode([query])[0]
       
        score = np.sum(query_emb * title_emb, axis=1) / np.linalg.norm(title_emb, axis=1)
        
        for i,s in enumerate(score):
          f.write(str(qid) + "," + str(dids[i]) + "," + str(s) + "," + str(top_score[i]) + "," + str(top_score2[i]) + "\n")
      if qid%100==0:
        if qid==1200 :
          flag=True
        if(flag):
          print(qid, score)
    except Exception as e:
      print(repr(e), qid)